In [128]:
import xarray as xr
import glob
import numpy as np
import os
from scipy.interpolate import griddata

In [62]:
platFilenames = glob.glob('/storage/ifremer-platform-meta/*/*/*_meta.nc')
print(len(platFilenames))

15267


In [61]:
for filename in platFilenames[0:10]:
    plat = xr.open_dataset('2900201_meta.nc')
    [x.strip(' ') for x in plat.variables['PARAMETER_SENSOR'].data.astype(str)]
    plat.variables['PARAMETER_RESOLUTION'].data.astype(float).tolist()
    pn = int(plat.variables['PLATFORM_NUMBER'].data)

In [37]:
plat.info()

xarray.Dataset {
dimensions:
	N_CONFIG_PARAM = 7 ;
	N_LAUNCH_CONFIG_PARAM = 7 ;
	N_MISSIONS = 1 ;
	N_PARAM = 3 ;
	N_POSITIONING_SYSTEM = 1 ;
	N_SENSOR = 3 ;
	N_TRANS_SYSTEM = 1 ;

variables:
	object DATA_TYPE() ;
		DATA_TYPE:long_name = Data type ;
		DATA_TYPE:conventions = Argo reference table 1 ;
	object FORMAT_VERSION() ;
		FORMAT_VERSION:long_name = File format version ;
	object HANDBOOK_VERSION() ;
		HANDBOOK_VERSION:long_name = Data handbook version ;
	object DATE_CREATION() ;
		DATE_CREATION:long_name = Date of file creation ;
		DATE_CREATION:conventions = YYYYMMDDHHMISS ;
	object DATE_UPDATE() ;
		DATE_UPDATE:long_name = Date of update of this file ;
		DATE_UPDATE:conventions = YYYYMMDDHHMISS ;
	object PLATFORM_NUMBER() ;
		PLATFORM_NUMBER:long_name = Float unique identifier ;
		PLATFORM_NUMBER:conventions = WMO float identifier : A9IIIII ;
	object PTT() ;
		PTT:long_name = Transmission identifier (ARGOS, ORBCOMM, etc.) ;
	object TRANS_SYSTEM(N_TRANS_SYSTEM) ;
		TRANS_SYSTEM:lo

In [19]:
[x.strip(' ') for x in plat.variables['PARAMETER_SENSOR'].data.astype(str)]

['CTD_PRES', 'CTD_TEMP', 'CTD_CNDC']

In [20]:
plat.variables['PARAMETER_RESOLUTION'].data.astype(float).tolist()

[0.1, 0.001, 0.001]

In [59]:
pn = int(plat.variables['PLATFORM_NUMBER'].data)

In [60]:
pn

2900201

In [ ]:
def init_basin_bak(self, basinFilename):
    nc = Dataset(basinFilename, 'r')

    assert nc.variables['LONGITUDE'].mask == True
    assert nc.variables['LATITUDE'].mask == True

    idx = np.nonzero(~nc.variables['BASIN_TAG'][:].mask)
    self.basin = nc.variables['BASIN_TAG'][:][idx].astype('i')
    self.coords = np.stack([nc.variables['LATITUDE'][idx[0]],
                       nc.variables['LONGITUDE'][idx[1]]]).T

def init_basin(self, basinFilename):
    nc = xr.open_dataset(basinFilename, 'r')
    idx = nc.variables['BASIN_TAG'][:]
    self.basin = nc.variables['BASIN_TAG'][:][idx].astype('i')
    self.coords = np.stack([nc.variables['LATITUDE'][idx[0]],
                       nc.variables['LONGITUDE'][idx[1]]]).T

In [141]:
basinFilename = './../basinmask_01.nc'
nc = xr.open_dataset(basinFilename)

In [165]:
type(nc)

xarray.core.dataset.Dataset

In [164]:
nc['BASIN_TAG'].shape

(168, 360)

In [152]:
%%timeit
nc.sel({"LONGITUDE": 0, "LATITUDE": 0}, method='nearest')

1.24 ms ± 65.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [161]:
int(nc.sel({"LONGITUDE": 0, "LATITUDE": 0}, method='nearest')['BASIN_TAG'])

1

In [143]:
df['BASIN_TAG'].unique()

array([10.,  2.,  1.,  3.,  0., 56.,  7.,  8.,  4., 12., 53.,  6.,  9.,
        5., 11.])

In [144]:
idx = nc.variables['BASIN_TAG'].data[:].flatten()
basin = nc.variables['BASIN_TAG'].data[:]
coords = np.stack([nc.variables['LATITUDE'].data[:].flatten(), nc.variables['LONGITUDE'].data[:].flatten()])
print(len(idx))


ValueError: all input arrays must have the same shape

In [147]:
basinFilename = './../basinmask_01.nc'
nc = xr.open_dataset(basinFilename)
df = nc.to_dataframe().dropna().reset_index()
basin = df['BASIN_TAG'].values
coords = np.stack([df['LATITUDE'].values, df['LONGITUDE'].values]).T
#df = df[df['BASIN_TAG'] != 0]
print(df.shape)
print(basin.shape)
print(coords.T.shape)

(41088, 3)
(41088,)
(2, 41088)


In [153]:
%%timeit

int(griddata(coords, basin, (lat, lon), method='nearest'))

24.7 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [247]:
fileName = '/home/tyler/Desktop/argo-database/test/test-files/aoml/1900722/profiles/SD1900722_004.nc'
profXR = xr.open_dataset(fileName)

In [272]:
myDict = profXR.to_dict()
print(myDict.keys())

dict_keys(['coords', 'attrs', 'dims', 'data_vars'])


In [248]:
variables = myDict['data_vars']
keys = variables.keys()

for key in keys:
    print(key)
    print(variables[key]['data'])

DATA_TYPE
b'Argo synthetic profile          '
FORMAT_VERSION
b'1.0 '
HANDBOOK_VERSION
b'1.2 '
REFERENCE_DATE_TIME
b'19500101000000'
DATE_CREATION
b'20190425093314'
DATE_UPDATE
b'20190425093314'
PLATFORM_NUMBER
[b'1900722 ']
PROJECT_NAME
[b'US ARGO PROJECT                                                 ']
PI_NAME
[b'STEPHEN RISER                                                   ']
STATION_PARAMETERS
[[b'PRES                                                            ', b'TEMP                                                            ', b'PSAL                                                            ', b'DOXY                                                            ']]
CYCLE_NUMBER
[4.0]
DIRECTION
[b'A']
DATA_CENTRE
[b'AO']
PARAMETER_DATA_MODE
[[b'D', b'D', b'D', b'R']]
PLATFORM_TYPE
[b'APEX                            ']
FLOAT_SERIAL_NO
[b'2596                            ']
FIRMWARE_VERSION
[b'012606                          ']
WMO_INST_TYPE
[b'846 ']
JULD
[datetime.datetime(2006,

In [262]:
[x.decode('utf-8') for x in variables['PARAMETER_DATA_MODE']['data'][0]]

['D', 'D', 'D', 'R']

In [277]:
myDict['dims']['N_PROF']

1

In [259]:
variables['DATA_TYPE']['data']

b'Argo synthetic profile          '

In [221]:
variables['DATA_TYPE']['data']

b'Argo synthetic profile          '

In [222]:
variables['DATA_TYPE']['FORMAT_VERSION']

KeyError: 'FORMAT_VERSION'

In [255]:
df['PARAMETER_DATA_MODE']

N_CALIB  N_LEVELS  N_PARAM  N_PROF
0        0         0        0         b'D'
                   1        0         b'D'
                   2        0         b'D'
                   3        0         b'R'
         1         0        0         b'D'
                   1        0         b'D'
                   2        0         b'D'
                   3        0         b'R'
         2         0        0         b'D'
                   1        0         b'D'
                   2        0         b'D'
                   3        0         b'R'
         3         0        0         b'D'
                   1        0         b'D'
                   2        0         b'D'
                   3        0         b'R'
         4         0        0         b'D'
                   1        0         b'D'
                   2        0         b'D'
                   3        0         b'R'
         5         0        0         b'D'
                   1        0         b'D'
                   

In [229]:
df = profXR.to_dataframe()
print(df.shape)
print(df.columns)

(284, 58)
Index(['DATA_TYPE', 'FORMAT_VERSION', 'HANDBOOK_VERSION',
       'REFERENCE_DATE_TIME', 'DATE_CREATION', 'DATE_UPDATE',
       'PLATFORM_NUMBER', 'PROJECT_NAME', 'PI_NAME', 'STATION_PARAMETERS',
       'CYCLE_NUMBER', 'DIRECTION', 'DATA_CENTRE', 'PARAMETER_DATA_MODE',
       'PLATFORM_TYPE', 'FLOAT_SERIAL_NO', 'FIRMWARE_VERSION', 'WMO_INST_TYPE',
       'JULD', 'JULD_QC', 'JULD_LOCATION', 'LATITUDE', 'LONGITUDE',
       'POSITION_QC', 'POSITIONING_SYSTEM', 'CONFIG_MISSION_NUMBER',
       'PARAMETER', 'SCIENTIFIC_CALIB_EQUATION',
       'SCIENTIFIC_CALIB_COEFFICIENT', 'SCIENTIFIC_CALIB_COMMENT',
       'SCIENTIFIC_CALIB_DATE', 'PROFILE_PRES_QC', 'PROFILE_TEMP_QC',
       'PROFILE_PSAL_QC', 'PROFILE_DOXY_QC', 'PRES', 'PRES_QC',
       'PRES_ADJUSTED', 'PRES_ADJUSTED_QC', 'PRES_ADJUSTED_ERROR', 'TEMP',
       'TEMP_QC', 'TEMP_dPRES', 'TEMP_ADJUSTED', 'TEMP_ADJUSTED_QC',
       'TEMP_ADJUSTED_ERROR', 'PSAL', 'PSAL_QC', 'PSAL_dPRES', 'PSAL_ADJUSTED',
       'PSAL_ADJUSTED_QC', 'PS

In [230]:
df[['PRES', 'TEMP', 'PSAL']]

PRES    TEMP       PSAL
N_CALIB N_LEVELS N_PARAM N_PROF                                
0       0        0       0          6.500000  13.461  35.112999
                 1       0          6.500000  13.461  35.112999
                 2       0          6.500000  13.461  35.112999
                 3       0          6.500000  13.461  35.112999
        1        0       0         10.500000  13.459  35.112999
                 1       0         10.500000  13.459  35.112999
                 2       0         10.500000  13.459  35.112999
                 3       0         10.500000  13.459  35.112999
        2        0       0         20.500000  13.460  35.112999
                 1       0         20.500000  13.460  35.112999
                 2       0         20.500000  13.460  35.112999
                 3       0         20.500000  13.460  35.112999
        3        0       0         30.299999  13.451  35.111000
                 1       0         30.299999  13.451  35.111000
                 2       0         30.299999  13.451  35.111000
                 3       0         30.299999  13.451  35.111000
        4        0       0         40.099998  13.447  35.112000
                 1       0         40.099998  13.447  35.112000
                 2       0         40.099998  13.447  35.112000
                 3       0         40.099998  13.447  35.112000
        5        0       0         50.200001  13.349  35.105000
                 1       0         50.200001  13.349  35.105000
                 2       0         50.200001  13.349  35.105000
                 3       0         50.200001  13.349  35.105000
        6        0       0         60.200001  13.156  35.092999
                 1       0         60.200001  13.156  35.092999
                 2       0         60.200001  13.156  35.092999
                 3       0         60.200001  13.156  35.092999
        7        0       0         70.000000  12.935  35.096001
                 1       0         70.000000  12.935  35.096001
...                                      ...     ...        ...
        63       2       0       1650.099976   2.793  34.584999
                 3       0       1650.099976   2.793  34.584999
        64       0       0       1699.900024   2.745  34.605000
                 1       0       1699.900024   2.745  34.605000
                 2       0       1699.900024   2.745  34.605000
                 3       0       1699.900024   2.745  34.605000
        65       0       0       1749.800049   2.694  34.625000
                 1       0       1749.800049   2.694  34.625000
                 2       0       1749.800049   2.694  34.625000
                 3       0       1749.800049   2.694  34.625000
        66       0       0       1800.099976   2.656  34.640999
                 1       0       1800.099976   2.656  34.640999
                 2       0       1800.099976   2.656  34.640999
                 3       0       1800.099976   2.656  34.640999
        67       0       0       1850.400024   2.624  34.654999
                 1       0       1850.400024   2.624  34.654999
                 2       0       1850.400024   2.624  34.654999
                 3       0       1850.400024   2.624  34.654999
        68       0       0       1899.500000   2.585  34.671001
                 1       0       1899.500000   2.585  34.671001
                 2       0       1899.500000   2.585  34.671001
                 3       0       1899.500000   2.585  34.671001
        69       0       0       1950.400024   2.547  34.686001
                 1       0       1950.400024   2.547  34.686001
                 2       0       1950.400024   2.547  34.686001
                 3       0       1950.400024   2.547  34.686001
        70       0       0       1998.199951   2.517  34.698002
                 1       0       1998.199951   2.517  34.698002
                 2       0       1998.199951   2.517  34.698002
                 3       0       1998.199951   2.517  34.698002

[284 ro